### Fine-tune model with Trainer API

In [ ]:
!pip install -U datasets
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("SetFit/mrpc")
checkpoint = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text1"], example["text2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from random import randint
def get_batch(tokenized_datasets, data_collator : DataCollatorWithPadding, batch_size, dataset_type = 'train',
              removed_keys = ['idx', 'label_text', 'text1', 'text2']):
  samples = tokenized_datasets[dataset_type]
  print(samples.num_rows)
  idx = randint(0, samples.num_rows - batch_size)
  samples = samples[idx:idx + batch_size]
  samples = {k:v for k, v in samples.items() if k not in removed_keys}
  batch = data_collator(samples)
  return batch

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/316 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# Define a TrainingArgument class that will contain all the hyperparameters the Trainer will use for training and evaluation
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval_use_gather_object=False,


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)
# Phần head của model cũ đã bị bỏ đi, để thêm vào head mới sử dụng cho tác vụ classification. Head này được khởi tạo ngẫu nhiên nên cần train.

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    processing_class = tokenizer
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ducanh2002add (ducanh2002add-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.582600
1000,0.359100


TrainOutput(global_step=1377, training_loss=0.4089976846089637, metrics={'train_runtime': 216.4738, 'train_samples_per_second': 50.833, 'train_steps_per_second': 6.361, 'total_flos': 419446300011600.0, 'train_loss': 0.4089976846089637, 'epoch': 3.0})

Chạy code vẫn tiến hành fine-tune nhưng sẽ không hiện các metric do chưa đặt eval_strategy trong TrainingArguments về 'step' (evaluate every eval_step) hoặc 'epoch' (evaluate at the end of each epoch)
và do chưa đưa vào Trainer hàm compute_metric() để tính metric trong quá trình evaluation.

### Evaluation

In [ ]:
# Get prediction
predictions = trainer.predict(
    tokenized_datasets['validation']
)

In [ ]:
print(predictions.predictions.shape) # Dự đoán của mô hình
print(predictions.label_ids.shape) # Nhãn thật sự

(408, 2)
(408,)


In [ ]:
preds = predictions.predictions.argmax(-1)
print(preds.shape)

(408,)


In [ ]:
accuracy = (preds == predictions.label_ids).mean()
print(accuracy)

0.8333333333333334


In [ ]:
!pip install -U evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.8 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("accuracy")
print(metric.compute(
    predictions = preds,
    references = predictions.label_ids
))

metric2 = evaluate.load("f1")
print(metric2.compute(
    predictions = preds,
    references = predictions.label_ids
))

{'accuracy': 0.8333333333333334}
{'f1': 0.8839590443686007}


In [ ]:
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  pred = logits.argmax(-1)
  accuracy_metric = evaluate.load("accuracy")
  f1_metric = evaluate.load("f1")
  accuracy = accuracy_metric.compute(
      predictions = pred,
      references = labels
  )
  f1 = f1_metric.compute(
      predictions = pred,
      references = labels
  )
  return {
      'accuracy': accuracy['accuracy'],
      'f1': f1['f1']
  }

In [ ]:
training_args = TrainingArguments(
    "test-trainer",
    eval_strategy = "steps",
    logging_steps = 100
)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)
trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
100,0.653900,0.545204,0.725490,0.826625
200,0.623000,0.504541,0.747549,0.835200
300,0.556100,0.483178,0.796569,0.853616
400,0.498800,0.536437,0.801471,0.870813
500,0.442900,0.429683,0.843137,0.884892
600,0.456700,0.409742,0.811275,0.867012
700,0.416600,0.443396,0.811275,0.862745
800,0.343700,0.669010,0.813725,0.872910
900,0.378300,0.564737,0.838235,0.889262
1000,0.253700,0.641376,0.808824,0.858182


TrainOutput(global_step=1377, training_loss=0.4028160009335675, metrics={'train_runtime': 261.1933, 'train_samples_per_second': 42.13, 'train_steps_per_second': 5.272, 'total_flos': 419446300011600.0, 'train_loss': 0.4028160009335675, 'epoch': 3.0})